### Load required libraries and define simulations settings path

In [1]:
import numpy as np
import scipy.sparse as sparse
import pickle

sim_folder = 'dos/mpcFullM1M2decoupled'

### MPC controller settings

In [ ]:
from pathlib import Path

# Plant dynamic model matrices
A = np.zeros((84,84))
B = np.identity(84)
nx,nu = A.shape[0], B.shape[1]

# MPC settings
# Prediction horizon
npred = 4
# Weighting matrices
Q = (0.2/npred)*sparse.block_diag([1]*nx)
R = (1/npred)*sparse.block_diag([1]*nu)
# Controller output constraints
dumin, dumax = -0.25*np.ones(nu), 0.25*np.ones(nu)
umin, umax = -2*np.ones(nu), 2*np.ones(nu)

# MPC dictionary
data = {'MPC':{'A':A,'B':B,'Q':Q,'R':R,
              'npred':npred,
               'dumin':dumin,'dumax':dumax,
               'umin':umin,'umax':umax}}
# Pickles MPC data into string representation
with open(Path(sim_folder)/'MPC.pickle','wb') as f:
    pickle.dump(data,f)

### Load DOS and define define folder with simulation settings

In [1]:
import logging

import dos
sim = dos.DOS(sim_folder,verbose=logging.INFO,show_timing=0)

NameError: name 'sim_folder' is not defined

In [3]:
sim.push()

INFO:DOS:Pushing configuration to server
INFO:M1:GMT
INFO:M2:GMT
INFO:wfs48:OP0
INFO:science:OP1


In [4]:
sim.init()

INFO:DOS:Initializing
INFO:M1:None
INFO:M2:None
INFO:wfs48:None
INFO:science:None


In [5]:
sim.terminate()

INFO:DOS:Terminating
INFO:M1:GMT deleted!
INFO:M2:GMT deleted!
INFO:wfs48:OpticalPath deleted!
INFO:science:OpticalPath deleted!


In [ ]:
# Start simulation
sim.start()

### Steady-state behavior

In [ ]:
M12_RBM = sim.logs.entries['wfs48']['M12_RBM'].timeSeries
M1_Txyz, M1_Rxyz = M12_RBM[1][:,:21], M12_RBM[1][:,21:42]
M2_Txyz, M2_Rxyz = M12_RBM[1][:,42:63], M12_RBM[1][:,63:]
scale = 1.0e6

print('M1 Txyz:')
for k in range(7):
    print('Seg#',k+1,':',scale*M1_Txyz[-1,0+(k*3):3+(k*3)])
    
print('\n M1 Rxyz:')
for k in range(7):
    print('Seg#',k+1,':',scale*M1_Rxyz[-1,0+(k*3):3+(k*3)])
    
print('\n M2 Txyz:')
for k in range(7):
    print('Seg#',k+1,':',scale*M2_Txyz[-1,0+(k*3):3+(k*3)])
    
print('\n M2 Rxyz:')
for k in range(7):
    print('Seg#',k+1,':',scale*M2_Rxyz[-1,0+(k*3):3+(k*3)])

### Plot simulation results
#### Mirror states

In [ ]:
# Plot mirror states
import matplotlib.pyplot as plt

plt.figure(figsize=(16,10))
plt.subplot(221)
plt.plot(M12_RBM[0],M1_Txyz,'.--')
plt.grid(True)
plt.ylabel('M1 Txyz state')
    
plt.subplot(222)
plt.plot(M12_RBM[0],M1_Rxyz,'.--')
plt.grid(True)
plt.ylabel('M1 Rxyz state')    

plt.figure(figsize=(16,5))
plt.subplot(223)
plt.plot(M12_RBM[0],M2_Txyz,'.--')
plt.grid(True)
plt.ylabel('M2 Txyz state')
    
plt.subplot(224)
plt.plot(M12_RBM[0],M2_Rxyz,'.--')
plt.grid(True)
plt.ylabel('M2 Rxyz state')   
plt.show()

In [ ]:
# Plot controller outputs
u2Txyz = sim.logs.entries['MPC']['M2 Txyz'].timeSeries
u2Rxyz = sim.logs.entries['MPC']['M2 Rxyz'].timeSeries
u1Txyz = sim.logs.entries['MPC']['M1 Txyz'].timeSeries
u1Rxyz = sim.logs.entries['MPC']['M1 Rxyz'].timeSeries

fig = plt.figure(figsize=(16,5))

plt1, plt2, plt3, plt4 = fig.add_subplot(221), fig.add_subplot(222), fig.add_subplot(223), fig.add_subplot(224)

for k in range(7):
    for kk in range(3):
        #Txyz
        plt1.plot(u1Txyz[1][k,kk,:],'.--')
        plt1.grid(True)
        plt1.set_ylabel('M1 Txyz cmd')
        plt1.autoscale(enable=True, axis='y')
        #Rxyz
        plt2.plot(u1Rxyz[1][k,kk,:],'.--')
        plt2.grid(True)
        plt2.set_ylabel('M1 Rxyz cmd')
        #Txyz
        plt3.plot(u2Txyz[1][k,kk,:],'.--')
        plt3.grid(True)
        plt3.set_ylabel('M2 Txyz cmd')
        plt3.autoscale(enable=True, axis='y')
        #Rxyz
        plt4.plot(u2Rxyz[1][k,kk,:],'.--')
        plt4.grid(True)
        plt4.set_ylabel('M2 Rxyz cmd')

plt.show()

In [ ]:
# Plot segment 1 mirror states
import matplotlib.pyplot as plt

M2_RBM = sim.logs.entries['wfs48']['M12_RBM'].timeSeries
M2_Txyz = M2_RBM[1][:,:3]
M2_Rxyz = M2_RBM[1][:,21:24]

plt.figure(figsize=(16,5))

plt.subplot(211)
plt.plot(M2_RBM[0],M2_Txyz,'.--')
plt.grid(True)
plt.ylabel('M2 Txyz state')
    
plt.subplot(212)
plt.plot(M2_RBM[0],M2_Rxyz,'.--')
plt.grid(True)
plt.ylabel('M2 Rxyz state')
    
plt.show()